In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('./models/')
from SIRH import *
import os
from SIRD_h import SIRD_h


In [2]:
df=pd.read_csv('hopitalized_and_infectious.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
n_hospitalized=np.array(df['hospitalized'])
n_infectious=np.array(df['n_infectious'])

In [3]:
# importing mobility from the csv file
df_mobility=pd.read_csv('mobility_bis.csv')
df_mobility.drop(columns=['Unnamed: 0'], inplace=True)
mobility=np.array(df_mobility['mobility'])

relier_les_points=[]
for i in range(len(mobility)): 
    if i + 7 < len(mobility): 
        if i % 7 ==0:
            relier_les_points.append(mobility[i])
        else: 
            decalage=i-7*(i//7)
            res = (1-decalage/7)*mobility[7*(i//7)] + (decalage/7)*mobility[7*(i//7)+7]

            relier_les_points.append(res)
    else:
        relier_les_points.append(mobility[i])
mobility_smoothed=np.array(relier_les_points)
data3D=np.array([n_hospitalized, n_infectious, mobility_smoothed])

In [4]:
s_0=1000000 -1
i_0=1
r_0=0
h_0=0
t=len(n_hospitalized)
dt=0.001
split=250
data_train=np.array([n_hospitalized[:split], n_infectious[:split], mobility_smoothed[:split]])
train_dates=[i for i in range(len(data_train))]

In [7]:
my_sirdh=SIRD_h()
my_sirdh.train(train_dates, data_train[0])
my_sirdh.choose_model(True, True)
my_sirdh.predict(7, 0.05)

AttributeError: 'SIRD_model_2' object has no attribute 'gamma_constant'